In [ ]:
!pip install feedparser # rss에서 xml태그별 정보 추출 (예: title, link, ...)
!pip install newspaper3k # 인터넷신문기사분석 (아래에 "Article()"을 사용하기 위함)
!pip install konlpy #한국어자연어처리/형태소 분석기 (주어진 문장에서 명사만 추출하는 용도로 여기서 사용)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6049 sha256=ae61993266c3cae613230dcd9374f07e80f6e33c11ef6dd9b4ea2748e77246c3
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 24.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13538 sha256=3b6695cdc477a58e1efc4ab1fec02ca48ef461ce7d4a2ecd059ecae4630a71ef
  Stored in directory: /root/.cache/pip/wheels/c8/d6/6c/384f58

In [ ]:
import feedparser
from newspaper import Article
from konlpy.tag import Okt
from collections import Counter # 명사를 konly로 추출한 후에 본문에 몇번 나오는지 확인 (tf계산용)
from bs4 import BeautifulSoup # 인터넷 기사 글에 존재할지 모르는 html 태그를 삭제하기 위해

In [ ]:
# 단계 1. 모든 RSS 파일을 돌아다니며 기사의 제목과 링크를 추출함
urls = ["http://rss.etnews.com/Section901.xml",
        "http://rss.etnews.com/Section902.xml",
        "http://rss.etnews.com/Section903.xml",
        "http://rss.etnews.com/Section904.xml"]

# rss 목록의 list안에 존재하는 모든 기사의 title, link를 list로 구성
def crawl_rss(urls):
    array_rss = [] # 여기에 모든 기사를 넣음
    title_rss = set() # 중복 기사 제거 (집합은 중복을 불허)
    for url in urls:
        print("[Crawl URL] ", url) # 현재 어디에 있는지 출력
        parse_rss = feedparser.parse(url) # 현재 url을 파싱한 후, 결과를 parse_rss에 저장
        for p in parse_rss.entries: # parse_rss에 있는 모든 entries/기사를 검색
            if p.title not in title_rss:
                array_rss.append({'title': p.title, 'link': p.link}) # 기사에서 제목/link 추출 후 리스트에 추가
                title_rss.add(p.title)
            else:
                print("Duplicated Article: ", p.title)

    return array_rss

list_articles = crawl_rss(urls)

print(list_articles)

[Crawl URL]  http://rss.etnews.com/Section901.xml
[Crawl URL]  http://rss.etnews.com/Section902.xml
Duplicated Article:  “최대 명절이 최악으로”… 태국 '송크란' 물축제 중 200명 사망
Duplicated Article:  새 아이패드 프로 코드명은?… “한국의 개나리”
Duplicated Article:  삼성전자 노조, 첫 쟁의행위
Duplicated Article:  “아이폰 16프로, 기본 스토리지 옵션 256GB·가격 999달러”
Duplicated Article:  [뉴스줌인]'과심위' 내실화 필요성 공감…공공 SW 제값받기 속도낼까
Duplicated Article:  “뭉쳐야 뜬다”…정보보호산업계 최초 합동 IR 열린다
Duplicated Article:  디플정위, 제5차 전체회의 개최···공공 AI 활성화 등 6개 정책 과제 발표
[Crawl URL]  http://rss.etnews.com/Section903.xml
[Crawl URL]  http://rss.etnews.com/Section904.xml
Duplicated Article:  “최대 명절이 최악으로”… 태국 '송크란' 물축제 중 200명 사망
Duplicated Article:  새 아이패드 프로 코드명은?… “한국의 개나리”
Duplicated Article:  삼성전자 노조, 첫 쟁의행위
Duplicated Article:  “아이폰 16프로, 기본 스토리지 옵션 256GB·가격 999달러”
Duplicated Article:  [뉴스줌인]'과심위' 내실화 필요성 공감…공공 SW 제값받기 속도낼까
Duplicated Article:  “뭉쳐야 뜬다”…정보보호산업계 최초 합동 IR 열린다
Duplicated Article:  디플정위, 제5차 전체회의 개최···공공 AI 활성화 등 6개 정책 과제 발표
Duplicated Article:  의협 “사태 풀 사람 대통령 뿐…의대 

In [ ]:
# 단계 2. list에 존재하는 모든 링크를 돌아다니면서 본문 text 긁어오기
# 하나의 url을 입력받아, 링크를 타고 들어가서 그 안의 title, text를 추출
# 언어는 한글로 기본 지정

def crawl_article(url, language='ko'):
    print('[Crawling Article] ', url)
    a = Article(url, language=language) # Article을 사용하여 그 url을 입력하고, 언어옵션지정 (한글 기본)
    a.download()
    a.parse()
    return a.title, preprocessing(a.text)

def preprocessing(text):
    text_article = BeautifulSoup(text, 'html5lib').get_text() # html 태그를 제거
    return text_article

for article in list_articles:
    _, text = crawl_article(article['link'])
    article['text'] = text

print(list_articles[0])

[Crawling Article]  https://www.etnews.com/20240417000343
[Crawling Article]  https://www.etnews.com/20240417000459
[Crawling Article]  https://www.etnews.com/20240417000456
[Crawling Article]  https://www.etnews.com/20240417000329
[Crawling Article]  https://www.etnews.com/20240417000314
[Crawling Article]  https://www.etnews.com/20240417000048
[Crawling Article]  https://www.etnews.com/20240416000361
[Crawling Article]  https://www.etnews.com/20240417000439
[Crawling Article]  https://www.etnews.com/20240417000345
[Crawling Article]  https://www.etnews.com/20240417000395
[Crawling Article]  https://www.etnews.com/20240417000305
[Crawling Article]  https://www.etnews.com/20240417000284
[Crawling Article]  https://www.etnews.com/20240417000049
[Crawling Article]  https://www.etnews.com/20240415000278
[Crawling Article]  https://www.etnews.com/20240417000365
[Crawling Article]  https://www.etnews.com/20240417000351
[Crawling Article]  https://www.etnews.com/20240417000340
[Crawling Arti

<ipython-input-5-d19b4267dca2>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text_article = BeautifulSoup(text, 'html5lib').get_text() # html 태그를 제거


[Crawling Article]  https://www.etnews.com/20240417000446
[Crawling Article]  https://www.etnews.com/20240417000445
[Crawling Article]  https://www.etnews.com/20240417000443
[Crawling Article]  https://www.etnews.com/20240417000442
[Crawling Article]  https://www.etnews.com/20240412000131
[Crawling Article]  https://www.etnews.com/20240412000124
[Crawling Article]  https://www.etnews.com/20240412000246
[Crawling Article]  https://www.etnews.com/20240412000027
[Crawling Article]  https://www.etnews.com/20240416000219
[Crawling Article]  https://www.etnews.com/20240416000195
[Crawling Article]  https://www.etnews.com/20240417000141
[Crawling Article]  https://www.etnews.com/20240412000214
[Crawling Article]  https://www.etnews.com/20240415000293
[Crawling Article]  https://www.etnews.com/20240415000301
[Crawling Article]  https://www.etnews.com/20240412000113
[Crawling Article]  https://www.etnews.com/20240415000433
[Crawling Article]  https://www.etnews.com/20240412000142
[Crawling Arti

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# 단계 3. 본문 text를 코사인 유사도를 활용하여 판단 (예: 첫번째 기사와 두번째 기사의 유사도 판단)
text_articles = [article['text'] for article in list_articles]
for i, x in enumerate(text_articles):
    print(i, x)
# vector = TfidfVectorizer(max_features=5000) # 주어진 문장을 벡터로 만드는 객체 생성
# tfidf_vectors = vector.fit_transform(text_articles) # fit_transform 함수를 이용하여 5000 사이즈 벡터 생성
# # tfidf_vector는 모든 기사를 5000사이즈 벡터로 만든 후 저장한 변수

# print(tfidf_vectors.shape) # (2024-04-16, 11h56m 기준 총 73개의 기사가 있음 (73, 5000))
# print(cosine_similarity(tfidf_vectors[0], tfidf_vectors[1])) # 첫번째 기사와 두번째 기사 비교

0 태국 최대 명절인 송크란 기간 동안 200명 이상이 사망했다. 송크란은 '지상 최대 물축제'로 불린다.



16일(현지시간) 태국 방콕포스트에 따르면 촌난 스리깨우 태국 공중보건부 장관은 송크란 축제 기간인 지난 11일부터 15일까지 206명이 숨지고 1,593명이 부상당했다고 밝혔다.



송크란은 태국의 명절로, 태국인들은 이 기간 불운을 씻는 의미로 불상이나 손에 정화수를 뿌리며 소원을 빈다.



이 풍습의 일환으로 송크란 연휴 기간 방콕, 치앙마이 등에서는 서로에게 물을 뿌리거나 물총을 쏘는 물 축제가 열린다. 대형 살수차와 코끼리까지 동원된다.



이때를 맞춰 태국을 방문하는 외국인 관광객도 연간 50만 명에 달한다. 유네스코는 지난해 12월 송크란을 무형문화유산으로 등재하기도 했다.



하지만 주요 도시에 많은 사람이 몰리고 축제 열기가 과열되면서 각종 사건·사고도 잇따른다.



올해 축제에서는 과속(43.2%)을 하거나 음주운전(23.9%)을 한 오토바이 사고가 다수 발생했다. 난투극이나 성범죄도 빈번하게 일어난다. 지난해 태국 시민사회단체가 진행한 설문조사에서 남녀 응답자 10명 중 6명은 송크란 기간 성추행을 경험했다고 답하기도 했다.



지난 14일에는 한 외국 관광객이 방콕 도심을 가로지르는 쌘쌥 운하에 빠진 물총을 꺼내려 강에 뛰어들었다가 익사하는 사건도 발생했다.



이번 행사 기간 수도 방콕에는 소방관·간호사를 포함한 공무원과 자원봉사자 1,600명이 배치됐다. 휴양지 파타야에는 경찰 600명을 투입했지만 인명 사고를 막지 못했다.



이원지 기자 news21g@etnews.com
1 다음달 출시 예정인 아이패드 새로운 모델의 코드명이 '개나리'인 것으로 전해져 화제다.



13일(현지시간) IT전문매체 나인투파이브맥은 애플이 최근 개발자용으로 배포한 iOS 17.5 베타 버전을 분석해 아이폰에 새로 도입될 기능과 애플의 세부 정보를 확인했다고 전했다.



iOS 17.5 베타는 빌드 번호(build number)

In [ ]:
print(text_articles)

["삼성전자가 2030년까지 추구할 디자인 지향점으로 본질(Essential)·혁신(Innovative)·조화(Harmonious)를 선언했다. 본질에 충실하고, 혁신에 도전하며, 삶과 조화를 이루는 제품을 디자인하겠다는 의지다.\n\n\n\n인공지능(AI) 시대를 맞아 새롭게 정립한 삼성전자의 디자인 철학이자, 삼성전자의 디자인 아이덴티티(DI) 5.0이다.\n\n노태문 모바일사업(MX) 사장 겸 디자인경영센터장은 이탈리아 '밀라노 디자인위크 2024' 개막에 앞서 레오나르도 다빈치 국립과학기술박물관 내 부지 레카발레리제(Le Cavallerizze)에서 장외전시(푸오리살로네)를 사전 공개하고, 이같이 발표했다.\n\n\n\n노 사장은 “사용자에 대한 이해와 공감이 기술 혁신과 동반됐을 때 비로소 의미있는 혁신 경험을 만들어내고 새로운 가능성을 열 수 있다고 생각한다”며 “이같은 디자인 철학을 바탕으로 변화하는 시대상을 반영해 본질을 추구하고 혁신적이고 조화를 이루는 디자인이라는 세 가지 디자인 방향성을 새로이 정립했다”고 설명했다.\n\n\n\n이어 “삶의 전 영역에 큰 영향을 끼치고 대전환을 불러온 AI 시대를 맞아 디자인도 이에 맞게 진화시켜 나가겠다”며 “고객이 어떤 삼성 제품을 선택하더라도 삶에서 더욱 의미있고 가치있는 경험을 누릴 수 있도록 노력을 다하겠다고” 덧붙였다.\n\n\n\n앞서 삼성전자는 2019년부터 2022년까지 '담대하라, 영혼과 교감하라(Be Bold. Resonate with Soul)'를 내걸고 과감한 혁신에 집중, 갤럭시폴드 등 혁신 제품을 선보였다. 새로운 철학에 따라 대대적 디자인 전환이 예상된다.\n\n\n\n삼성전자는 밀라노 디자인 위크에서 사람과 기술이 공존하는 미래의 모습을 제시한 '공존의 미래'를 주제로 전시했다. 디스플레이와 센서, 몰입형 미디어 아트 등 기법을 이용해 현대적 기술과 예술적 가치를 조화롭게 선보여 사람과 사람, 사람과 제품, 나아가 사회적 가치와 환경까지 고려한 이상적 공존을 표현했다.\n\n삼

In [ ]:
# 단계 4. 글(단어, 문장)을 입력 받아 코사인 유사도로 가장 유사한 문서 1개 검출
query = [input()]
target = vector.transform(query)
similarities = [(i, cosine_similarity(target, tfidf_vector)) for i, tfidf_vector in enumerate(tfidf_vectors)]
index, max_similarity = max(similarities, key = lambda x: x[1])
print('-' * 50)
print("유사도 가장 높은 기사의 인덱스: ", index)
print("유사도: %5.2f%%" %(100*max_similarity.item()))
print("유사도가 가장 높은 기사의 제목: ", list_articles[index]['title'])
print("유사도가 가장 높은 기사의 링크: ", list_articles[index]['link'])


삼성
--------------------------------------------------
유사도 가장 높은 기사의 인덱스:  10
유사도: 14.10%
유사도가 가장 높은 기사의 제목:  美, 삼성 반도체 보조금 64억달러 확정
유사도가 가장 높은 기사의 링크:  https://www.etnews.com/20240415000436
